In [10]:
# 1. Install Transformers (if not already installed)
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
!pip install transformers datasets accelerate peft -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.5 MB/s eta 0:00:00:00:0100:01


#Importing required libraries

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
!pip install transformers accelerate peft

In [13]:
import pandas as pd
import numpy as np

print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)


Pandas version: 2.2.3
Numpy version: 1.26.4


In [14]:
import os
print(os.path.exists("/kaggle/input/clinical-reasoning-dataset/train.csv"))


True


In [34]:
import pandas as pd
from datasets import Dataset

train_df = pd.read_csv("/kaggle/input/clinical-reasoning-dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/clinical-reasoning-dataset/test.csv")


In [35]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Master_Index         400 non-null    object 
 1   County               400 non-null    object 
 2   Health level         400 non-null    object 
 3   Years of Experience  300 non-null    float64
 4   Prompt               400 non-null    object 
 5   Nursing Competency   400 non-null    object 
 6   Clinical Panel       400 non-null    object 
 7   Clinician            400 non-null    object 
 8   GPT4.0               400 non-null    object 
 9   LLAMA                400 non-null    object 
 10  GEMINI               400 non-null    object 
 11  DDX SNOMED           399 non-null    object 
dtypes: float64(1), object(11)
memory usage: 37.6+ KB


In [36]:
train_df.columns

Index(['Master_Index', 'County', 'Health level', 'Years of Experience',
       'Prompt', 'Nursing Competency', 'Clinical Panel', 'Clinician', 'GPT4.0',
       'LLAMA', 'GEMINI', 'DDX SNOMED'],
      dtype='object')

In [37]:
train_df_2 = train_df[['Prompt', 'Clinician']].dropna().rename(columns={'Prompt': 'input', 'Clinician': 'output'})

In [38]:
train_dataset_2 = Dataset.from_pandas(train_df_2)

In [39]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

def preprocess(example):
    return tokenizer(
        ["clinical prompt: " + x for x in example["input"]],
        text_target=example["output"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_train_2 = train_dataset_2.map(preprocess, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [40]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small").to("cpu")

In [48]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch

training_args = TrainingArguments(
    output_dir="./finetuned-flan-t5-reasoning",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    learning_rate=5e-5,
    fp16=torch.cuda.is_available(),
    logging_steps=10
)


from transformers import Trainer, DataCollatorForSeq2Seq

# Re-initialize the trainer with same config
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)


/tmp/ipykernel_35/2291468295.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [49]:
test_df = pd.read_csv("/kaggle/input/clinical-reasoning-dataset/test.csv")  # update with correct path
test_df = test_df[['Prompt']].dropna().reset_index(drop=True)

# Wrap into Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df.rename(columns={'Prompt': 'input'}))


In [50]:
def preprocess(example):
    return tokenizer(
        ["clinical prompt: " + x for x in example["input"]],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_test_dataset = test_dataset.map(preprocess, batched=True)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [54]:
# Generate predictions from raw prompts using model.generate
from tqdm import tqdm

# Add prediction column to test DataFrame
generated_responses = []

for prompt in tqdm(test_df['Prompt']):
    input_text = "clinical prompt: " + prompt
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(model.device)
    
    output_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        num_beams=4,
        early_stopping=True
    )
    
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    generated_responses.append(decoded)

# Attach to test DataFrame and save
test_df['generated_response'] = generated_responses
test_df.to_csv("flan_t5_predictions.csv", index=False)



100%|██████████| 100/100 [00:25<00:00,  3.87it/s]


In [55]:
# Attach to test DataFrame and save
test_df.head(5)

,Prompt,generated_response
0,i am a nurse with 2 years of experience in gen...,bronchitis
1,i am a nurse with 22 years of experience in ge...,to the right nostrils
2,i am a nurse working in a national referral ho...,if it’s a serious disease
3,i am a nurse working in a dispensaries and pri...,twitching
4,i am a nurse working in a health centres in ka...,i should just refer this boy and notify the pu...
